In [1]:
import numpy as np
import pandas as pd
from keras.models import Model, load_model

Using Theano backend.


# Load candidate models

In [5]:
submission_1 = "tests/fixtures/20200731_predictor.h5"
submission_2 = "tests/fixtures/20200730_predictor.h5"
submission_3 = "tests/fixtures/20200729_predictor.h5"

In [6]:
from keras.constraints import Constraint

class Positive(Constraint):

    def __call__(self, w):
        return K.abs(w)

In [7]:
model_1 = load_model(submission_1, custom_objects={"Positive": Positive})
model_2 = load_model(submission_2, custom_objects={"Positive": Positive})
model_3 = load_model(submission_3, custom_objects={"Positive": Positive})

# Load data

## Latest data

In [8]:
# DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
DATA_URL = "tests/fixtures/OxCGRT_latest.csv"
latest_df = pd.read_csv(DATA_URL,
                        parse_dates=['Date'],
                        encoding="ISO-8859-1",
                        error_bad_lines=False)

/Users/m_754337/workspace/esp-demo/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
# Handle regions
latest_df["RegionName"].fillna('', inplace=True)
# Replace CountryName by CountryName / RegionName
# np.where usage: if A then B else C
latest_df["CountryName"] = np.where(latest_df["RegionName"] == '',
                                    latest_df["CountryName"],
                                    latest_df["CountryName"] + ' / ' + latest_df["RegionName"])

## Latest data before submission

In [22]:
submission_date = np.datetime64("2020-07-31")

In [23]:
latest_df[latest_df.CountryName == "United States"].Date.max()

Timestamp('2020-08-28 00:00:00')

In [24]:
snapshot_df = latest_df[latest_df.Date <= submission_date]

In [25]:
snapshot_df[snapshot_df.CountryName == "United States"].Date.max()

Timestamp('2020-07-31 00:00:00')

## NPIs since submission

In [26]:
NPI_COLUMNS = ['C1_School closing',
               'C2_Workplace closing',
               'C3_Cancel public events',
               'C4_Restrictions on gatherings',
               'C5_Close public transport',
               'C6_Stay at home requirements',
               'C7_Restrictions on internal movement',
               'C8_International travel controls',
               'H1_Public information campaigns',
               'H2_Testing policy',
               'H3_Contact tracing']

In [31]:
npis_df = latest_df[latest_df.Date > submission_date]

In [28]:
npis_df[npis_df.CountryName == "United States"].Date.min()

Timestamp('2020-08-01 00:00:00')

In [29]:
npis_df = npis_df[["CountryName", "Date"] + NPI_COLUMNS]

In [30]:
npis_df[npis_df.CountryName == "United States"].head(30)

,CountryName,Date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing
43352,United States,2020-08-01,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43353,United States,2020-08-02,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43354,United States,2020-08-03,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43355,United States,2020-08-04,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43356,United States,2020-08-05,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43357,United States,2020-08-06,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43358,United States,2020-08-07,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43359,United States,2020-08-08,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43360,United States,2020-08-09,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0
43361,United States,2020-08-10,3.0,2.0,2.0,4.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0


# Ranking

## Day 1

### NPIs

In [69]:
start_date = np.datetime64("2020-08-01")
start_date

numpy.datetime64('2020-08-01')

In [70]:
end_date = start_date + np.timedelta64(3, 'D')
end_date

numpy.datetime64('2020-08-04')

In [71]:
day_1_npis_df = latest_df[(latest_df.Date >= start_date) & (latest_df.Date <= end_date)]

In [72]:
day_1_npis_df.Date.unique()

array(['2020-08-01T00:00:00.000000000', '2020-08-02T00:00:00.000000000',
       '2020-08-03T00:00:00.000000000', '2020-08-04T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [73]:
# day_1_npis_df = npis_df[npis_df.Date <= day1]

In [74]:
day_1_npis_df[day_1_npis_df.CountryName == "Aruba"][["CountryName", "RegionName", "Date", "ConfirmedCases"] + NPI_COLUMNS].head(30)

,CountryName,RegionName,Date,ConfirmedCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing
213,Aruba,,2020-08-01,121.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0
214,Aruba,,2020-08-02,121.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0
215,Aruba,,2020-08-03,122.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0
216,Aruba,,2020-08-04,124.0,0.0,1.0,0.0,4.0,0.0,1.0,1.0,3.0,2.0,2.0,1.0


### Prediction

In [ ]:
preds = model_1.predict([context_input, action_input])